In [ ]:
!pip install -qU gigachain==0.2.6 gigachain_community==0.2.6 gigachain-cli==0.0.25 langgraph==0.2.19 langchain-community==0.2.16 langchain-openai==0.1.23  python-pptx==0.6.23 python-pptx-interface==0.0.12 python-dotenv==1.0.1 duckduckgo-search==6.2.4 faiss-cpu

In [1]:
import getpass
import os
import requests
import json
import time

from langchain.chat_models.gigachat import GigaChat
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory, ConversationSummaryMemory, VectorStoreRetrieverMemory
from langchain.chains import ConversationChain
from langchain_community.embeddings import GigaChatEmbeddings

from langchain.schema import HumanMessage, SystemMessage
from langchain.tools import tool

import faiss
from langchain_community.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

from pydantic import BaseModel

In [2]:
from dotenv import load_dotenv
# Load environment variables
load_dotenv('.env')

True

# 1. Memory

Define your own class implementing a simple LLM-based chatbot. You need to use at least three memory types (langchain.memory), which are set as one argument in the ```init``` definition. If the memory type has any parameters, you also need to define them as arguments in the ```init``` definition. You also need to define a ```run``` method implementing the main conversation loop, and a ```print_memory``` method to print out what exactly the memory consists of.

In [21]:
class SimpleChatBot:
    def __init__(self, llm, memory_type):
        #TODO
    def _respond(self, user_input):
        #TODO
    
    def print_memory(self):
        #TODO

    def run(self):
        print("This is a simple chat bot:")
        #TODO


Now let's check how it works with each type of memory

In [22]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga = GigaChat(credentials=giga_key, model="GigaChat", timeout=30, verify_ssl_certs=False)
chat = SimpleChatBot(giga, 'summary', memory_window_size=2)
chat.run()
chat.print_memory()

This is a simple chat bot:
Конечно! Екатеринбург - это большой город в России, расположенный на Урале. Это третий по величине город в стране после Москвы и Санкт-Петербурга. Он известен своей промышленностью, образованием и культурой. В городе много музеев, театров и других культурных учреждений. Также здесь есть несколько университетов, включая Уральский федеральный университет имени Б. Н. Ельцина. Кроме того, Екатеринбург является важным транспортным узлом, соединяющим Европу и Азию.
В центре Екатеринбурга можно посетить множество достопримечательностей. Например, можно прогуляться по Плотинке - это исторический район города, где находится старинная плотина, построенная еще в XIX веке. Также стоит посетить Храм-на-Крови, который был построен на месте дома Ипатьева, где была убита царская семья Романовых. Еще одно интересное место - это Исторический сквер, где находятся памятники архитектуры и музеи. Кроме того, в центре города есть много магазинов, кафе и ресторанов, где можно отдохн

In [ ]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga = GigaChat(credentials=giga_key, model="GigaChat", timeout=30, verify_ssl_certs=False)
chat = SimpleChatBot(giga, 'window', memory_window_size=2)
chat.run()
chat.print_memory()

In [ ]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga = GigaChat(credentials=giga_key, model="GigaChat", timeout=30, verify_ssl_certs=False)
chat = SimpleChatBot(giga, 'simple', memory_window_size=2)
chat.run()
chat.print_memory()

Please make a short report abount differences between used memory types

Report:

# 2. Using tools and agents

## 2.1 Using tools and API

Create your own tool based on the langchain.tools library to interact with a public OpenWeather API. This tool will receive data from the API and return it as a readable result for the user.


OpenWeather API URL: https://api.openweathermap.org/data/2.5/weather?q={city}&appid={openweather_key}&units=metric 

[How to get OpenWeather API key](https://docs.google.com/document/d/1vbi8QKqMZqZoCReIzpmEB_2mHsrbmXPlyGngE3jeDDw/edit)


In [3]:
from langchain.tools import Tool
from langchain.agents import create_gigachat_functions_agent, AgentExecutor
import requests

openweather_key = os.environ.get("OPENWEATHER_API_KEY")

#TODO
def get_wheather(city: str) -> str:
    #TODO

class OpenWeatherAPITool:
    def __init__(self, llm, agent_function):
        self.llm = llm
        self.agent_function = agent_function

    def run(self, user_input: str):
        #TODO


Let's check it

In [30]:

giga_key = os.environ.get("SB_AUTH_DATA")
giga_pro = GigaChat(credentials=giga_key, model="GigaChat-Pro", timeout=30, verify_ssl_certs=False)

openwheatertool = OpenWeatherAPITool(giga_pro, get_wheather)
user_input = "Какая погода сейчас в Ставрополе?"
openwheatertool.run(user_input)

{'input': 'Какая погода сейчас в Ставрополе?',
 'output': 'В Ставрополе сейчас ясно, температура 18.16°С.'}

## 2.2. Multi agents

Create a multi-agent system where each agent is responsible for a specific task in the travel planning process. For example, one agent is responsible for searching for flights, another for booking hotels, and a third for finding the weather at the destination.

Requirements:

- Use three or more GigaChat-based agents to interact with each other.
- The first agent is responsible for searching for flights (using ```get_url_booking_tickets``` function).
- The second agent is responsible for booking hotels (using ```get_url_booking_hotels``` function).
- The third agent collects weather information for the destination (using a real API, such as OpenWeather). You can use the function from the previous task (for simplify, here you can give a current weather, not a forecast for the specific date)

In [23]:
from langchain.tools import Tool
from langchain.agents import create_gigachat_functions_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import requests
import json
import datetime
from dateutil import parser

openweather_key = os.environ.get("OPENWEATHER_API_KEY")

def get_geoid(city: str) -> str:
    url_base = 'https://suggest-maps.yandex.ru/suggest-geo'
    params = {'search_type': 'tune', 'v': '9', 'results': 1, 'lang': 'ry_RU', 'callback': 'json'}
    params['part'] = city
    r = requests.get(url_base, params=params)
    if r.ok:
        r_text = r.text
        r_json = r_text[5: len(r_text)-1]
        res_json = json.loads(r_json)
        res = res_json['results'][0]['geoid']
    else:
        res = ''
    return str(res)


#TODO
def get_url_booking_hotels(date_in_str: str, date_out_str: str, city: str) -> str:
    #TODO
    date_in = parser.parse(date_in_str)
    date_out = parser.parse(date_out_str)
    if date_in is None:
        date_in = datetime.datetime.now()
    if date_out is None:
        date_out = datetime.datetime.now() + datetime.timedelta(days=1)
    geoid = get_geoid(city)
    url = 'https://travel.yandex.ru/hotels/search/?' 
    params = {'adults': '2', 'checkinDate': date_in.strftime('%Y-%m-%d'), 'checkoutDate': date_out.strftime('%Y-%m-%d'), 'childrenAges': '0', 'geoId': geoid}
    for item in params:
        url += '&' + item + '=' + params[item]
    return f'Here is your URL for booking: {url} in {city} on {date_in_str} / {date_out_str}'


#TODO
def get_url_booking_tikets(city_from: str, city_to: str, date_in_str: str, date_out_str: str) -> str:
    #TODO
    date_in = parser.parse(date_in_str)
    date_out = parser.parse(date_out_str)
    if date_in is None:
        date_in = datetime.datetime.now()
    if date_out is None:
        date_out = datetime.datetime.now() + datetime.timedelta(days=1)
    fromid = get_geoid(city_from)
    toid = get_geoid(city_to)
    url = 'https://travel.yandex.ru/avia/search/result/?' 
    params = {'adults_seats': '2', 'fromId': 'c' + fromid, 'klass': 'economy', 'oneway': '2', 'return_date': date_out.strftime('%Y-%m-%d'), 'toId': 'c' + toid, 'when': date_in.strftime('%Y-%m-%d')}
    for item in params:
        url += '&' + item + '=' + params[item]
    return f'Here is your url for tickets ordering: {url} from {city_from} to {city_to} on {date_in_str} / {date_out_str}'



class MultiAgent:
    def __init__(self, llm, agent_function_wheater, agent_function_hotels, agent_function_tickets):
        self.llm = llm
        self.agent_function_wheater = agent_function_wheater
        self.agent_function_hotels = agent_function_hotels
        self.agent_function_tickets = agent_function_tickets

    def _create_agent(self, tools, system_prompt):
        #TODO
        return AgentExecutor

    def run(self, user_input: str):
        #TODO
        return answer


In [24]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga_pro = GigaChat(credentials=giga_key, model="GigaChat-Pro", timeout=30, verify_ssl_certs=False)

traveler = MultiAgent(giga_pro, get_wheather, get_url_booking_hotels, get_url_booking_tikets)
user_input = "Организуй поездку в Санкт-Петербурге на 10 дней с 15.11.2024 - отель, самолет, погода"
answer = traveler.run(user_input)
print(answer)

Организуй поездку в Санкт-Петербурге на 10 дней с 15.11.2024 - отель, самолет, погода
Вот Ваша ссылка для бронирования отеля: [https://travel.yandex.ru/hotels/search/?&adults=2&checkinDate=2024-11-15&checkoutDate=2024-11-24&childrenAges=0&geoId=2](https://travel.yandex.ru/hotels/search/?&adults=2&checkinDate=2024-11-15&checkoutDate=2024-11-24&childrenAges=0&geoId=2) в Санкт-Петербурге с 15 по 24 ноября 2024 года.
Вот Ваша ссылка для заказа билетов: https://travel.yandex.ru/avia/search/result/?&adults_seats=2&fromId=c213&klass=economy&oneway=2&return_date=2024-11-24&toId=c2&when=2024-11-15 от Москвы до Санкт-Петербурга на 15.11.2024 / 24.11.2024.
Погода в Санкт-Петербурге сейчас облачная с температурой 8.92°C.
